In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:
import matplotlib.pyplot as plt
import cfe.regression as rgsn
import numpy as np
from scipy.stats import gaussian_kde as kde
from cfe.df_utils import use_indices

Missing dependencies for OracleDemands.


In [18]:
def init_label(label, ax):
    txt = ax.text(label.x, label.y, label.text, fontsize=label.font_size, rotation=label.rotation, transform = ax.transData)
    bbox = txt.get_window_extent(renderer=ax.get_figure().canvas.get_renderer())
    label.bbox = bbox
    txt.remove()
    
def first_pass_clearing(labels, ax):
    for i,label1 in enumerate(labels):
        for j,label2 in enumerate(labels):
            if check_collision(label1,label2):
                assert not(label1.postion_strict and label2.position_strict), "Overlap of strictly positioned labels: "+label1.text+", and "+label2.text
                
                if label1.position_strict:
                    labels.pop(j)
                elif label2.position_strict:
                    labels.pop(i)

                elif label1.font_size >= label2.font_size:
                    labels.pop(j)
                else:
                    labels.pop(i)
        
def update_labels(labels, ax):
    for label in labels:
        if label.position_strict:
            continue
        if label.target_font_size <= label.font_size:
            continue

        label.font_size+=1
        txt = ax.text(label.x, label.y, label.text, fontsize=label.font_size, rotation=label.rotation, 
                      transform=ax.transData)

        bbox = txt.get_window_extent(renderer=ax.get_figure().canvas.get_renderer())

        label.bbox = bbox

        for label2 in labels:
            if check_collision(label,label2):
                label.font_size-=1
        txt.remove()
def check_collision(label1, label2):
    return label1.bbox.overlaps(label2.bbox)
def add_labels(labels,ax):
    for label in labels:
        ax.text(label.x, label.y, label.text, fontsize=label.font_size, rotation=label.rotation, transform=ax.transData, color=label.color)

In [19]:
from abc import abstractmethod, ABC
class EngelLabel:
    def __init__(
        self,
        target_font_size,
        position,
        font_size = 0,
        position_strict = False,
        color = 'black',
        deg_rotation = 0
                ):
        self.bbox = None
        self.target_font_size = target_font_size
        self.position_strict = position_strict
        self.color = color
        
class BaseEngelPie(ABC):
    def plot_axis(self,ax):
        ax.arrow(0,0,1,0,shape='full',head_width=.05,length_includes_head=True)
        ax.annotate(r"$\log x$", xy=(1,0),color='red')    
    def annotate(self):
        ...
    def label(self):
        ...
    @abstractmethod
    def graph(
        self, 
        post_graphing_fn = lambda ax: ax
             ):
        plot_axis(ax)
        label(ax)
        post_graphing_fn(ax)
        return plt

In [26]:
class RgsnEngelPie(BaseEngelPie):
    # data needs to be
    def __init__(self, data_path = None, data = None):
        assert (data_path or data)
        
        self.data_path = data_path
        self.data = data
        self.r = rgsn.read_pickle(self.data_path) if data_path else data
        self.xbar = None
        self.x99 = None
        self.Y = None
        self.p = None
        self.shares = None
        self.stats_computed = False

    def compute_statistics(self):
        predicted_expenditures = self.r.predicted_expenditures()
        grouped = predicted_expenditures.groupby(['i','t','m'])
        self.xbar = grouped.sum()
        
        self.x99 = self.xbar.quantile(0.99)
        
        self.Y = np.geomspace(1, self.x99, 100)
        
        y = self.r.y
        unstacked = y.unstack('j')
        booleaned = unstacked>0
        floated = booleaned+0.
        self.p = floated.mean()
        
        grouped_j = predicted_expenditures.groupby('j')
        grouped_j_sum = grouped_j.sum()
        self.shares = grouped_j_sum * self.p
        
        self.stats_computed = True
    def graph(self,fn=lambda ax: ax):
        if not self.stats_computed: self.compute_statistics(self)

        fig, ax = plt.subplots()

        labels = list()
        for label in self.shares.index.tolist():
            labels.append(EngelLabel(TODO: magnitude of label,position,
        
        ax.pie(self.shares, labels=self.shares.index.tolist(), rotatelabels=False)
        
        for i in range(len(self.Y)-1,0,-1):
            ax.pie(self.r.expenditures(self.Y[i])*self.p,radius=np.log(self.Y[i])/np.log(self.x99))
        
        ax.arrow(0,0,1,0,shape='full',head_width=.05,length_includes_head=True)
        ax.annotate(r"$\log x$", xy=(1,0),color='red')
        return plt
        # return AbstractEngelPie.graph(self)

In [21]:
# Example usage:
file_path = '../data/preferred.rgsn'
chart = RgsnEngelPie(file_path)

In [22]:
chart.compute_statistics()

In [27]:
plt = chart.graph()

TypeError: RgsnEngelPie.graph() missing 1 required positional argument: 'fn'